In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [3]:
df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')

In [4]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [5]:
df.shape

(3066766, 19)

In [6]:
len(df.columns)

19

In [7]:
df.dtypes

VendorID                          int64
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
airport_fee                     float64
dtype: object

In [8]:
df['duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']

In [9]:
df['duration'] = df['duration'].dt.total_seconds() / 60

In [10]:
df['duration'].std()

42.594351241920904

In [11]:
df_without_outliers_ration = len(df[df['duration'].between(1, 60)]) / len(df)
df_without_outliers_ration

0.9812202822125979

In [12]:
df = df[df['duration'].between(1, 60)]

In [13]:
catigorical = ['DOLocationID', 'PULocationID']
df[catigorical] = df[catigorical].astype('str')

In [14]:
train_dicts = df[catigorical].to_dict(orient='records')

In [15]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [16]:
X_train

<3009173x515 sparse matrix of type '<class 'numpy.float64'>'
	with 6018346 stored elements in Compressed Sparse Row format>

In [17]:
Y_train = df['duration'].values

In [18]:
lr = LinearRegression().fit(X_train, Y_train)

In [19]:
rmse = mean_squared_error(Y_train, lr.predict(X_train), squared=False)
rmse

7.649261027919939

In [24]:
def fun(parquet_link):
    df = pd.read_parquet(parquet_link)
    df['duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
    df['duration'] = df['duration'].dt.total_seconds() / 60
    df = df[df['duration'].between(1, 60)]
    catigorical = ['DOLocationID', 'PULocationID']
    df[catigorical] = df[catigorical].astype('str')
    train_dicts = df[catigorical].to_dict(orient='records')
    return train_dicts, df['duration'].values

In [25]:
val_dict, Y_val = fun('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [26]:
val_dict[:5]

[{'DOLocationID': '163', 'PULocationID': '142'},
 {'DOLocationID': '26', 'PULocationID': '132'},
 {'DOLocationID': '145', 'PULocationID': '161'},
 {'DOLocationID': '236', 'PULocationID': '148'},
 {'DOLocationID': '244', 'PULocationID': '137'}]

In [27]:
X_val = dv.transform(val_dict)

In [28]:
pred = lr.predict(X_val)

In [29]:
rmse = mean_squared_error(Y_val, pred, squared=False)
rmse

7.811832638273232